<a href="https://colab.research.google.com/github/noobosapien/colab-gists/blob/main/initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers scikit-learn numpy sentencepiece rank_bm25 datasets pandas faiss-cpu accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 25.3 MB/s eta 0:00:00


In [4]:
from transformers import DPRContextEncoderTokenizer, DPRContextEncoder
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from huggingface_hub import notebook_login



In [8]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [9]:
notebook_login()


Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [11]:
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
documents = [
    "The IPL 2024 was a thrilling season with unexpected results.",
    "The invention of the printing press revolutionized communication in the 15th century.",
    "Neo4j is a graph database platform for connected data applications.",
    "ChatGPT is a large language model developed by OpenAI.",
    "Dense Passage Retrieval (DPR) is a state-of-the-art technique for information retrieval."
]


def encode_documents(documents):
  inputs = tokenizer(documents, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
    outputs = model(**inputs)
  return outputs.pooler_output.numpy()

document_embeddings = encode_documents(documents)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
def retrieve_documents(query, num_results=3):
  inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
    query_embedding = model(**inputs).pooler_output.numpy()
  similarity_scores = cosine_similarity(
      query_embedding, document_embeddings
  ).flatten()
  top_indices = similarity_scores.argsort()[-num_results:][::-1]
  top_docs = [
      (documents[i], similarity_scores[i])
      for i in top_indices
  ]

  return top_docs

In [17]:
query = "What is dense passage retrieval?"
doc = retrieve_documents(query)
print(doc)

[('Dense Passage Retrieval (DPR) is a state-of-the-art technique for information retrieval.', np.float32(0.77766144)), ('Neo4j is a graph database platform for connected data applications.', np.float32(0.6700113)), ('ChatGPT is a large language model developed by OpenAI.', np.float32(0.6029494))]


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)
model = T5ForConditionalGeneration.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
device

device(type='cuda')

In [6]:
query = "What are the benefits of solar energy?"
retrieved_passages = """
Solar energy is a renewable resource and reduces electricity bills.
It is environmentally friendly and helps combat climate change.
Solar panels require minimal maintenance and have a long lifespan.
"""

In [ ]:
def generate_response(query, retrieved_passages):
  input_text = f"Answer this question based on the provided context: {query} Context: {retrieved_passages}"

  inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
  with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=300,
        num_beams=3,
        early_stopping=True
    )